# Car Evaluation

1. Title: Car Evaluation Database

2. Sources:
   (a) Creator: Marko Bohanec
   (b) Donors: Marko Bohanec   (marko.bohanec@ijs.si)
               Blaz Zupan      (blaz.zupan@ijs.si)
   (c) Date: June, 1997

3. Past Usage:

   The hierarchical decision model, from which this dataset is derived, was first presented in M. Bohanec and V. Rajkovic: Knowledge acquisition and explanation for multi-attribute decision making. In 8th Intl Workshop on Expert Systems and their Applications, Avignon, France. pages 59-78, 1988.

   Within machine-learning, this dataset was used for the evaluation of HINT (Hierarchy INduction Tool), which was proved to be able to completely reconstruct the original hierarchical model. This, together with a comparison with C4.5, is presented in B. Zupan, M. Bohanec, I. Bratko, J. Demsar: Machine learning by
   function decomposition. ICML-97, Nashville, TN. 1997 (to appear)

4. Relevant Information Paragraph:

   Car Evaluation Database was derived from a simple hierarchical decision model originally developed for the demonstration of DEX (M. Bohanec, V. Rajkovic: Expert system for decision making. Sistemica 1(1), pp. 145-157, 1990.). The model evaluates cars according to the following concept structure:

   ```text
   CAR                      car acceptability
   . PRICE                  overall price
   . . buying               buying price
   . . maint                price of the maintenance
   . TECH                   technical characteristics
   . . COMFORT              comfort
   . . . doors              number of doors
   . . . persons            capacity in terms of persons to carry
   . . . lug_boot           the size of luggage boot
   . . safety               estimated safety of the car
   ```

   Input attributes are printed in lowercase. Besides the target concept (CAR), the model includes three intermediate concepts:  PRICE, TECH, COMFORT. Every concept is in the original model related to its lower level descendants by a set of examples (for these examples sets see http://www-ai.ijs.si/BlazZupan/car.html).

   The Car Evaluation Database contains examples with the structural information removed, i.e., directly relates CAR to the six input attributes: buying, maint, doors, persons, lug_boot, safety.

   Because of known underlying concept structure, this database may be particularly useful for testing constructive induction and structure discovery methods.

5. Number of Instances: 1728
   (instances completely cover the attribute space)

6. Number of Attributes: 6

7. Attribute Values:

   ```text
   buying       v-high, high, med, low
   maint        v-high, high, med, low
   doors        2, 3, 4, 5-more
   persons      2, 4, more
   lug_boot     small, med, big
   safety       low, med, high
   ```

8. Missing Attribute Values: none

9. Class Distribution (number of instances per class)

   ```text
   class      N          N[%]
   -----------------------------
   unacc     1210     (70.023 %) 
   acc        384     (22.222 %) 
   good        69     ( 3.993 %) 
   v-good      65     ( 3.762 %) 
   ```

In [1]:
# Import required dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Import data
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m13/lesson_3/datasets/car.csv"
df = pd.read_csv(file_path)
df.head()

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc


In [3]:
# Check the value_counts of the target column
df["class"].value_counts()

class
unacc    1210
acc       384
good       69
vgood      65
Name: count, dtype: int64

## Preprocess the data

In [4]:
# Check the data types
df.dtypes

buying      object
maint       object
doors       object
persons     object
lug_boot    object
safety      object
class       object
dtype: object

In [5]:
# Get the target variable (the "class" column)
y = df["class"]
y

0       unacc
1       unacc
2       unacc
3       unacc
4       unacc
        ...  
1723     good
1724    vgood
1725    unacc
1726     good
1727    vgood
Name: class, Length: 1728, dtype: object

In [6]:
# Get the features (everything except the "class" column)
X = df.copy()
X = X.drop(columns="class")
X.head()

,buying,maint,doors,persons,lug_boot,safety
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med


In [7]:
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
# Since the target column is an object, we need to convert the data to numerical classes
# Encode the y data
# Create an instance of the label encoder
le = LabelEncoder()

# Fit and transform the y training and testing data using the label encoder
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)
y_train_encoded

array([2, 2, 0, ..., 0, 2, 0])

In [9]:
# Remember that all of the columns in the DataFrame are objects
# Use a OneHotEncoder to convert the training data to numerical values
ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int')
X_train_encoded = pd.DataFrame(data=ohe.fit_transform(X_train), columns=ohe.get_feature_names_out())
X_train_encoded

,buying_high,buying_low,buying_med,buying_vhigh,maint_high,maint_low,maint_med,maint_vhigh,doors_2,doors_3,...,doors_5more,persons_2,persons_4,persons_more,lug_boot_big,lug_boot_med,lug_boot_small,safety_high,safety_low,safety_med
0,0,0,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,1
1,1,0,0,0,0,1,0,0,1,0,...,0,1,0,0,0,0,1,0,0,1
2,1,0,0,0,1,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,1
3,0,0,1,0,0,0,0,1,0,1,...,0,0,1,0,1,0,0,0,0,1
4,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1291,0,0,1,0,0,0,1,0,0,1,...,0,0,0,1,0,1,0,1,0,0
1292,0,0,1,0,0,1,0,0,0,0,...,1,0,0,1,1,0,0,0,0,1
1293,1,0,0,0,0,1,0,0,0,0,...,1,0,0,1,0,1,0,1,0,0
1294,0,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1


In [10]:
# Encode the test data
X_test_encoded = pd.DataFrame(data=ohe.transform(X_test), columns=ohe.get_feature_names_out())
X_test_encoded

,buying_high,buying_low,buying_med,buying_vhigh,maint_high,maint_low,maint_med,maint_vhigh,doors_2,doors_3,...,doors_5more,persons_2,persons_4,persons_more,lug_boot_big,lug_boot_med,lug_boot_small,safety_high,safety_low,safety_med
0,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,1,0,0
1,0,0,1,0,0,1,0,0,1,0,...,0,0,1,0,0,1,0,0,0,1
2,1,0,0,0,1,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,1
3,0,1,0,0,1,0,0,0,0,0,...,1,0,1,0,0,1,0,0,0,1
4,0,0,1,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,0,0,1,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
428,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,1
429,0,0,1,0,1,0,0,0,0,0,...,1,0,0,1,0,1,0,0,1,0
430,0,0,1,0,0,1,0,0,0,1,...,0,0,0,1,0,0,1,1,0,0


## Model and Fit to a Logistic Regression Classifier

In [11]:
# Create the logistic regression classifier model with a random_state of 1
lr_model = LogisticRegression(random_state=1)

# Fit the model to the training data
lr_model.fit(X_train_encoded, y_train_encoded)

LogisticRegression(random_state=1)

In [12]:
# Validate the model by checking the model accuracy with model.score
print('Train Accuracy: %.3f' % lr_model.score(X_train_encoded, y_train_encoded))
print('Test Accuracy: %.3f' % lr_model.score(X_test_encoded, y_test_encoded))

Train Accuracy: 0.925
Test Accuracy: 0.917


## Model and Fit to a Support Vector Machine

In [13]:
# Create the support vector machine classifier model with a 'poly' kernel
svm_model = SVC(kernel='poly')

# Fit the model to the training data
svm_model.fit(X_train_encoded, y_train_encoded)

SVC(kernel='poly')

In [14]:
# Validate the model by checking the model accuracy with model.score
print('Train Accuracy: %.3f' % svm_model.score(X_train_encoded, y_train_encoded))
print('Test Accuracy: %.3f' % svm_model.score(X_test_encoded, y_test_encoded))

Train Accuracy: 0.998
Test Accuracy: 0.977


## Model and Fit to a KNN Model

In [15]:
# Create the KNN model with 9 neighbors
knn_model = KNeighborsClassifier(n_neighbors=9)

# Fit the model to the training data
knn_model.fit(X_train_encoded, y_train_encoded)

KNeighborsClassifier(n_neighbors=9)

In [16]:
# Validate the model by checking the model accuracy with model.score
print('Train Accuracy: %.3f' % knn_model.score(X_train_encoded, y_train_encoded))
print('Test Accuracy: %.3f' % knn_model.score(X_test_encoded, y_test_encoded))

Train Accuracy: 0.960
Test Accuracy: 0.919


## Model and Fit to a Decision Tree Classifier

In [17]:
# Create the decision tree classifier model
dt_model = DecisionTreeClassifier()

# Fit the model to the training data
dt_model.fit(X_train_encoded, y_train_encoded)

DecisionTreeClassifier()

In [18]:
# Validate the model by checking the model accuracy with model.score
print('Train Accuracy: %.3f' % dt_model.score(X_train_encoded, y_train_encoded))
print('Test Accuracy: %.3f' % dt_model.score(X_test_encoded, y_test_encoded))

Train Accuracy: 1.000
Test Accuracy: 0.965


## Model and Fit to a Random Forest Classifier

In [19]:
# Create the random forest classifier model
# with n_estimators=128 and random_state=1
rf_model = RandomForestClassifier(n_estimators=128, random_state=1)

# Fit the model to the training data
rf_model.fit(X_train_encoded, y_train_encoded)

RandomForestClassifier(n_estimators=128, random_state=1)

In [20]:
# Validate the model by checking the model accuracy with model.score
print('Train Accuracy: %.3f' % rf_model.score(X_train_encoded, y_train_encoded))
print('Test Accuracy: %.3f' % rf_model.score(X_test_encoded, y_test_encoded))

Train Accuracy: 1.000
Test Accuracy: 0.961


# Build a Pipeline Function Instead

In [21]:
# Import data
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m13/lesson_3/datasets/car.csv"
df = pd.read_csv(file_path)
# Get the target variable (the "class" column)
y = df["class"]
# Get the features (everything except the "class" column)
X = df.copy().drop(columns="class")
# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import SVR

def run_pipeline(model, X_train, X_test, y_train, y_test):
    # Since the target column is an object, we need to convert the data to numerical classes
    # Encode the y data
    # Create an instance of the label encoder
    le = LabelEncoder()

    # Fit and transform the y training and testing data using the label encoder
    y_train_encoded = le.fit_transform(y_train)
    y_test_encoded = le.transform(y_test)

    # Remember that all of the columns in the DataFrame are objects
    # Use a OneHotEncoder to convert the training data to numerical values
    ohe = OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int')
    X_train_encoded = pd.DataFrame(data=ohe.fit_transform(X_train), columns=ohe.get_feature_names_out())
    X_test_encoded = pd.DataFrame(data=ohe.transform(X_test), columns=ohe.get_feature_names_out())

    # Fit the model to the training data
    model.fit(X_train_encoded, y_train_encoded)

    # Validate the model by checking the model accuracy with model.score
    train_accuracy = model.score(X_train_encoded, y_train_encoded)
    test_accuracy = model.score(X_test_encoded, y_test_encoded)
    return train_accuracy, test_accuracy


models = [LinearRegression(), KNeighborsRegressor(n_neighbors=9), RandomForestRegressor(n_estimators=128, random_state=1), ExtraTreesRegressor(n_estimators=128, random_state=1), 
          AdaBoostRegressor(n_estimators=128, random_state=1), SVR(C=1.0, epsilon=0.2),
          LogisticRegression(random_state=1), SVC(kernel='poly'), KNeighborsClassifier(n_neighbors=9), DecisionTreeClassifier(), 
          RandomForestClassifier(n_estimators=128, random_state=1)]
for m in models:
    train_accuracy, test_accuracy = run_pipeline(m, X_train, X_test, y_train, y_test)
    print(f"Model: {m}")
    print(f"Train Accuracy: {train_accuracy}")
    print(f"Test Accuracy: {test_accuracy}")
    print("")

Model: LinearRegression()
Train Accuracy: 0.2749015164813692
Test Accuracy: 0.25430914606294575

Model: KNeighborsRegressor(n_neighbors=9)
Train Accuracy: 0.6697864201365087
Test Accuracy: 0.5618859267117651

Model: RandomForestRegressor(n_estimators=128, random_state=1)
Train Accuracy: 0.9844477476685747
Test Accuracy: 0.8609407692137679

Model: ExtraTreesRegressor(n_estimators=128, random_state=1)
Train Accuracy: 1.0
Test Accuracy: 0.8125524916013438

Model: AdaBoostRegressor(n_estimators=128, random_state=1)
Train Accuracy: 0.3304247823404788
Test Accuracy: 0.27711066535756335

Model: SVR(epsilon=0.2)
Train Accuracy: 0.7499699949470742
Test Accuracy: 0.5952850032165627

Model: LogisticRegression(random_state=1)
Train Accuracy: 0.9251543209876543
Test Accuracy: 0.9166666666666666

Model: SVC(kernel='poly')
Train Accuracy: 0.9984567901234568
Test Accuracy: 0.9768518518518519

Model: KNeighborsClassifier(n_neighbors=9)
Train Accuracy: 0.9598765432098766
Test Accuracy: 0.918981481481481